<a href="https://colab.research.google.com/github/frank-morales2020/Cloud_curious/blob/master/Copy_of_model_load_work_transformer_demo_fp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Model environmenmt

In [1]:
!pip install colab-env -q

In [ ]:
import colab_env
import os

access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")

from huggingface_hub import login

login(
  token=access_token_write, # ADD YOUR TOKEN HERE
  add_to_git_credential=True
)

In [3]:
!pip install transformers datasets torch -q

In [ ]:
!pip show transformers
print()
!pip show datasets
print()
!pip show torch

## training

In [5]:
import random
import math
from datasets import Dataset
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
from tqdm import tqdm
from geopy.point import Point  # Import Point object
import warnings
warnings.filterwarnings("ignore")
import torch
import re

# Define airport codes and names
airports = [
    # North America
    "ATL", "LAX", "ORD", "DFW", "DEN", "JFK", "SFO", "LAS", "SEA", "CLT",
    "MIA", "PHL", "EWR", "BOS", "PHX", "MCO", "IAH", "DAL", "FLL", "DTW",
    "MSP", "IAD", "BWI", "SLC", "SAN", "MDW", "TPA", "PDX", "BNA", "STL",
    # Europe
    "LHR", "CDG", "AMS", "FRA", "MAD", "BCN", "FCO", "MUC", "DUB", "ZRH",
    # Asia
    "HND", "PVG", "ICN", "SIN", "KUL", "BKK", "DXB", "DEL", "HKG", "NRT",
    # Caribbean
    "CUN", "PUJ", "SJU", "MBJ", "NAS",
    # South America
    "GRU", "BOG", "EZE", "SCL", "LIM",
    # Australia & New Zealand
    "SYD", "MEL", "BNE", "PER", "AKL"
]

airport_names = {
    "ATL": "Hartsfield-Jackson Atlanta International Airport",
    "LAX": "Los Angeles International Airport",
    "ORD": "Chicago O'Hare International Airport",
    "DFW": "Dallas/Fort Worth International Airport",
    "DEN": "Denver International Airport",
    "JFK": "John F. Kennedy International Airport",
    "SFO": "San Francisco International Airport",
    "LAS": "Harry Reid International Airport",
    "SEA": "Seattle-Tacoma International Airport",
    "CLT": "Charlotte Douglas International Airport",
    "MIA": "Miami International Airport",
    "PHL": "Philadelphia International Airport",
    "EWR": "Newark Liberty International Airport",
    "BOS": "Logan International Airport",
    "PHX": "Phoenix Sky Harbor International Airport",
    "MCO": "Orlando International Airport",
    "IAH": "George Bush Intercontinental Airport",
    "DAL": "Dallas Love Field",
    "FLL": "Fort Lauderdale-Hollywood International Airport",
    "DTW": "Detroit Metropolitan Airport",
    "MSP": "Minneapolis-Saint Paul International Airport",
    "IAD": "Washington Dulles International Airport",
    "BWI": "Baltimore/Washington International Airport",
    "SLC": "Salt Lake City International Airport",
    "SAN": "San Diego International Airport",
    "MDW": "Midway International Airport",
    "TPA": "Tampa International Airport",
    "PDX": "Portland International Airport",
    "BNA": "Nashville International Airport",
    "STL": "St. Louis Lambert International Airport",
    # Europe
    "LHR": "London Heathrow Airport",
    "CDG": "Charles de Gaulle Airport",
    "AMS": "Amsterdam Airport Schiphol",
    "FRA": "Frankfurt Airport",
    "MAD": "Adolfo Suárez Madrid–Barajas Airport",
    "BCN": "Barcelona–El Prat Airport",
    "FCO": "Leonardo da Vinci–Fiumicino Airport",
    "MUC": "Munich Airport",
    "DUB": "Dublin Airport",
    "ZRH": "Zurich Airport",
    # Asia
    "HND": "Haneda Airport",
    "PVG": "Shanghai Pudong International Airport",
    "ICN": "Incheon International Airport",
    "SIN": "Singapore Changi Airport",
    "KUL": "Kuala Lumpur International Airport",
    "BKK": "Suvarnabhumi Airport",
    "DXB": "Dubai International Airport",
    "DEL": "Indira Gandhi International Airport",
    "HKG": "Hong Kong International Airport",
    "NRT": "Narita International Airport",
    # Caribbean
    "CUN": "Cancún International Airport",
    "PUJ": "Punta Cana International Airport",
    "SJU": "Luis Muñoz Marín International Airport",
    "MBJ": "Sangster International Airport",
    "NAS": "Lynden Pindling International Airport",
    # South America
    "GRU": "São Paulo–Guarulhos International Airport",
    "BOG": "El Dorado International Airport",
    "EZE": "Ministro Pistarini International Airport",
    "SCL": "Arturo Merino Benítez International Airport",
    "LIM": "Jorge Chávez International Airport",
    # Australia & New Zealand
    "SYD": "Sydney Airport",
    "MEL": "Melbourne Airport",
    "BNE": "Brisbane Airport",
    "PER": "Perth Airport",
    "AKL": "Auckland Airport"
}

In [6]:
from transformers import AutoTokenizer
tokenizer_name = 'gpt2'
#tokenizer_name = 'meta-llama/Meta-Llama-3-8B'
#tokenizer_name = 'mistralai/Mistral-7B-Instruct-v0.3'
pad_token = '<pad>'

# --- Load Tokenizer and Add Pad Token ---
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': pad_token})
# Ensure pad_token_id is within the vocabulary size
pad_token_id = tokenizer.pad_token_id
if pad_token_id >= tokenizer.vocab_size:
    pad_token_id = tokenizer.vocab_size - 1

In [ ]:
from datasets import load_dataset
flight_plan_dataset = load_dataset("frankmorales2020/flight_plan_waypoints")
flight_plan_dataset['train'][7]

In [8]:
import torch

def preprocess_flight_plan(example):
    """
    Preprocesses a single flight plan example.
    """
    input_text = example['input']

    # Tokenize the input text
    tokens = tokenizer(
        input_text,
        truncation=True,
        max_length=sequence_length,
        padding="max_length",
        return_tensors="pt",
    )

    # Ensure input_ids and attention_mask are tensors and have the correct shape
    for key in ['input_ids', 'attention_mask']:
        tokens[key] = tokens[key].squeeze(0)  # Remove extra dimension if present

    # Extract relevant data
    tokens['num_waypoints'] = example['label']

    # Convert waypoints to tensor here
    tokens['waypoints'] = torch.tensor(example['waypoints'][1:-1], dtype=torch.float32)  # Intermediate waypoints

    # Handle out-of-vocabulary tokens (clip input_ids)
    tokens['input_ids'] = tokens['input_ids'].clip(0, tokenizer.vocab_size - 1)

    return tokens

In [9]:
def custom_collate_fn(batch):
    """
    Collates a batch of flight plan examples.
    """
    # Find the maximum number of waypoints in the batch
    max_waypoints = max(len(item['waypoints']) for item in batch)

    # Pad waypoints with [0.0, 0.0] to the maximum length
    padded_waypoints = [
        torch.cat([item['waypoints'], torch.zeros((max_waypoints - len(item['waypoints']), 2), dtype=torch.float32)])
        for item in batch
    ]

    # Stack the data (using torch.stack directly)
    collated_batch = {
        'input_ids': torch.stack([item['input_ids'] for item in batch]),
        'attention_mask': torch.stack([item['attention_mask'] for item in batch]),
        'waypoints': torch.stack(padded_waypoints),
        'num_waypoints': torch.tensor([item['num_waypoints'] for item in batch]),
    }
    return collated_batch

In [ ]:
!pip install transformers datasets torch geopy -q

from transformers import AutoTokenizer
import json
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer
from datasets import load_dataset, DatasetDict
import numpy as np
import os
from tqdm import tqdm
import random
from geopy.geocoders import Nominatim
from warnings import filterwarnings
filterwarnings('ignore')

# Set environment variables for CUDA
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ['TORCH_USE_CUDA_DSA'] = '1'

# --- Hyperparameters ---
batch_size = 4  # Reduced batch size
sequence_length = 256
embedding_dimension = 256
num_heads = 8
feed_forward_dimension = 1024
num_encoder_layers = 6
dropout_probability = 0.3  # Increased dropout
learning_rate = 2e-5  # Reduced learning rate
num_epochs = 1  # Increased epochs
tokenizer_name = 'gpt2'
pad_token = '<pad>'
waypoint_count_penalty_weight = 10.0  # Increased penalty weight
max_waypoints = 10  # Maximum number of waypoints for length embedding



# --- Load Tokenizer and Add Pad Token ---
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': pad_token})
pad_token_id = tokenizer.pad_token_id
if pad_token_id >= tokenizer.vocab_size:
    pad_token_id = tokenizer.vocab_size - 1

# *** Add airport codes to tokenizer vocabulary ***
for airport_code in airports:
    if airport_code not in tokenizer.vocab:  # Check if already in vocab
        tokenizer.add_tokens([airport_code])

# --- Load and Preprocess Flight Plan Data ---
flight_plan_dataset = load_dataset("frankmorales2020/flight_plan_waypoints")

# Split into train and validation sets
flight_plan_dataset = flight_plan_dataset['train'].train_test_split(test_size=0.2, seed=42)
flight_plan_dataset = DatasetDict({
    'train': flight_plan_dataset['train'],
    'validation': flight_plan_dataset['test']
})

# --- Define the Transformer Model (SimpleTransformer with length embedding) ---
class SimpleTransformer(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_heads, ff_hidden_dim, num_layers, block_size, dropout, max_waypoints):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.pos_embedding = nn.Embedding(block_size, embed_dim)
        self.length_embedding = nn.Embedding(max_waypoints + 1, embed_dim)  # Length embedding
        encoder_layer = nn.TransformerEncoderLayer(embed_dim, num_heads, ff_hidden_dim, dropout)
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers)
        self.linear_out = nn.Linear(embed_dim, 2)  # Output layer for latitude and longitude
        self.block_size = block_size
        self.dropout = nn.Dropout(dropout)

    def forward(self, input_ids, attention_mask, num_waypoints):
        positions = torch.arange(0, input_ids.size(1), dtype=torch.long, device=input_ids.device).unsqueeze(0)
        pos_embeddings = self.pos_embedding(positions)

        # Get length embedding
        #num_waypoints = num_waypoints.clip(0, self.length_embedding.num_embeddings - 1)  # Clamp num_waypoints  Removed for Training
        length_embed = self.length_embedding(num_waypoints)
        # Expand length embedding to match input sequence length
        length_embed = length_embed.unsqueeze(1).repeat(1, input_ids.shape[1], 1)

        # Combine embeddings
        embeddings = self.dropout(self.embedding(input_ids) + pos_embeddings + length_embed)

        encoder_output = self.encoder(embeddings, src_key_padding_mask=(attention_mask == 0))

        # Reshape output for waypoint coordinates
        output = self.linear_out(encoder_output)
        return output

# --- Haversine Distance Function ---
import torch

def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Calculate the Haversine distance between two points.
    """
    # Convert to radians
    lat1, lon1, lat2, lon2 = map(torch.deg2rad, [lat1, lon1, lat2, lon2])

    # Apply Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = torch.sin(dlat / 2)**2 + torch.cos(lat1) * torch.cos(lat2) * torch.sin(dlon / 2)**2
    c = 2 * torch.arcsin(torch.sqrt(a))
    r = 6371  # Radius of earth in kilometers. Use 3956 for miles
    return c * r

# --- Loss Function with Waypoint Count Penalty ---
def custom_loss(pred, target, num_waypoints_target):
    distance_loss = torch.mean(haversine_distance(
            pred[:, :num_waypoints_target.max(), 0],
            pred[:, :num_waypoints_target.max(), 1],
            target[:, :num_waypoints_target.max(), 0],
            target[:, :num_waypoints_target.max(), 1],
        ))

    # Penalty for exceeding the target number of waypoints
    extra_waypoints = torch.clamp(pred.shape[1] - num_waypoints_target.max(), min=0)

    # *** Modified Penalty Calculation (Squared Difference and Increased Weight) ***
    waypoint_count_penalty_weight = 50.0  # Increased from 10.0
    waypoint_count_penalty = waypoint_count_penalty_weight * (extra_waypoints.float() ** 2)

    # Total loss
    total_loss = distance_loss + waypoint_count_penalty
    return total_loss
# --- Instantiate the model ---
vocab_size = tokenizer.vocab_size
block_size = sequence_length
model = SimpleTransformer(vocab_size, embedding_dimension, num_heads,
                          feed_forward_dimension, num_encoder_layers,
                          block_size, dropout_probability, max_waypoints)


# Update the embedding layer of the model to accommodate the new tokens
model.embedding = nn.Embedding(tokenizer.vocab_size, embedding_dimension)


# --- Move model to CUDA device ---
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)


# --- Create DataLoaders ---
processed_dataset = flight_plan_dataset.map(preprocess_flight_plan, batched=False)  # Process individually
train_loader = DataLoader(processed_dataset['train'], batch_size=batch_size, shuffle=True, collate_fn=custom_collate_fn)
validation_loader = DataLoader(processed_dataset['validation'], batch_size=batch_size, shuffle=False,
                              collate_fn=custom_collate_fn)

# --- Optimizer ---
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# --- Training Loop ---
def train(model, dataloader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    for batch_idx, batch in enumerate(dataloader):
        # 1. Data Preparation
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        num_waypoints_target = batch['num_waypoints'].to(device)

        # Clamp num_waypoints_target and ensure correct data type and device:
        num_waypoints_target = num_waypoints_target.clip(0, model.length_embedding.num_embeddings - 1)
        num_waypoints_target = num_waypoints_target.type(torch.LongTensor).to(device)

        # Extract intermediate waypoints as target
        target_waypoints = batch['waypoints'].to(device) # Get the padded waypoints here

        # 2. Model Prediction
        optimizer.zero_grad()
        predicted_waypoints = model(input_ids, attention_mask, num_waypoints_target)

        # 3. Loss Calculation (with penalty for exceeding waypoint count)
        loss = criterion(predicted_waypoints, target_waypoints, num_waypoints_target)

        # 4. Optimization
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    return total_loss / len(dataloader)

# --- Training Execution ---
if __name__ == "__main__":

    for epoch in tqdm(range(num_epochs), desc="Training/Epochs"):
        train_loss = train(model, train_loader, optimizer, custom_loss, device)
        print('\n')
        print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {train_loss:.4f}")

## Model Deployment To HF

In [ ]:
!pip install --upgrade huggingface_hub -q
from huggingface_hub import notebook_login, create_repo, upload_file

In [ ]:
from huggingface_hub import notebook_login, create_repo, upload_file

repo_name = "frankmorales2020/FlightPlan_Transformer_LLM"

# --- Automatically generate README.md ---
readme_content = f"""
# Flight Plan Transformer

This repository contains a Transformer model trained to generate flight plan waypoints based on textual input. The model was trained using the "frankmorales2020/flight_plan_waypoints" dataset from Hugging Face Datasets.

## Model Description

The model is a decoder-only Transformer architecture implemented using PyTorch and the `transformers` library. It takes textual input describing the flight (e.g., "Calculate the waypoints from SIN to CUN. Departure: 2024-06-19, Aircraft: Airbus A320, Weather: Partly Cloudy") and predicts a sequence of waypoints representing the flight path.

## Intended Use

This model can be used to:

- **Generate flight plans:** Given flight details, the model can predict waypoints for the route.
- **Flight planning assistance:** It can serve as a tool to aid in flight planning, providing suggestions for waypoints.

## Limitations

- **Accuracy:** The model's accuracy is limited by the quality and coverage of the training data. It may not be perfectly accurate in all scenarios.
- **Real-world applicability:** This model is a research prototype and should not be used for actual flight navigation without thorough validation and safety checks.

## How to Use

1. **Install the necessary libraries:**
Use code with caution
bash pip install transformers datasets torch

2. **Load the model and tokenizer:**
Use code with caution
python from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("{repo_name}") model = AutoModelForSeq2SeqLM.from_pretrained("{repo_name}")

3. **Generate waypoints:**
Use code with caution
python def generate_flight_plan(model, tokenizer, start_text, max_new_tokens=200):

# ... (Your generation code from the original script) ...
4. **Example usage:**
Use code with caution
python query = "Calculate the waypoints from SIN to CUN. Departure: 2024-06-19, Aircraft: Airbus A320, Weather: Partly Cloudy" generated_plan = generate_flight_plan(model, tokenizer, start_text=query, max_new_tokens=256) print(generated_plan)

## Training Data

The model was trained on the "frankmorales2020/flight_plan_waypoints" dataset from Hugging Face Datasets. This dataset contains flight plans with corresponding waypoints.

## Evaluation

The model's performance was evaluated on a held-out portion of the training data. (Include specific metrics or evaluation results if available)

## Acknowledgements

- Thanks to Hugging Face for providing the `transformers` and `datasets` libraries.
- Thanks to frankmorales2020 for creating and sharing the flight plan dataset.

## Contact
Frank Morales, BEng, MEng, SMIEEE
Boeing Associate Technical Fellow
Linkedin: https://www.linkedin.com/in/frank-morales1964/
"""

# Write README.md to file
with open("README.md", "w") as f:
    f.write(readme_content)

In [ ]:
import os
access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")
token=access_token_write

In [ ]:
from huggingface_hub import HfApi
api = HfApi()
api.get_token_permission(token=access_token_write)
repo_id = 'frankmorales2020/FlightPlan_Transformer_LLM'
api.delete_repo(repo_id=repo_id)

In [ ]:
from huggingface_hub import HfApi, HfFolder
import os
import shutil

# --- Push to Hugging Face Hub ---

repo_name = "frankmorales2020/FlightPlan_Transformer_LLM"
create_repo(repo_name, private=False, exist_ok=True)

# --- Push the model directly from memory ---
api = HfApi()
api.upload_file(
    path_or_fileobj="README.md",
    path_in_repo="README.md",
    repo_id=repo_name,
    repo_type="model",
)

# Save the model and tokenizer to a temporary directory
model_save_dir = "flight_plan_transformer_model"
os.makedirs(model_save_dir, exist_ok=True)

# 1. Create a configuration dictionary for your model
config = {
    "vocab_size": tokenizer.vocab_size,
    "embed_dim": embedding_dimension,  # Use your embedding dimension variable
    "num_heads": num_heads,  # Use your num_heads variable
    "ff_hidden_dim": feed_forward_dimension,  # Use your feed_forward_dimension variable
    "num_layers": num_encoder_layers,  # Use your num_encoder_layers variable
    "block_size": block_size,  # Use your block_size variable
    "dropout": dropout_probability  # Use your dropout_probability variable
}

# 2. Save the configuration to config.json
with open(os.path.join(model_save_dir, "config.json"), "w") as f:
    json.dump(config, f)

# Save the model's state_dict instead of using save_pretrained
torch.save(model.state_dict(), os.path.join(model_save_dir, 'pytorch_model.bin'))
tokenizer.save_pretrained(model_save_dir) # This is fine as tokenizer is from Hugging Face


# Upload the model files, including config.json and pytorch_model.bin
upload_file(
    path_or_fileobj=os.path.join(model_save_dir, "config.json"),
    path_in_repo="config.json",
    repo_id=repo_name,
)
upload_file(
    path_or_fileobj=os.path.join(model_save_dir, "pytorch_model.bin"),
    path_in_repo="pytorch_model.bin",
    repo_id=repo_name,
)

# Upload other files (e.g., tokenizer files)
for filename in os.listdir(model_save_dir):
    if filename not in ["config.json", "pytorch_model.bin"]:  # Exclude already uploaded files
        upload_file(
            path_or_fileobj=os.path.join(model_save_dir, filename),
            path_in_repo=filename,
            repo_id=repo_name,
        )


# Remove the temporary directory (optional)
shutil.rmtree(model_save_dir)

print(f"Model pushed to Hugging Face Hub: {repo_name}")

## Model Evaluation

In [ ]:
from transformers import AutoTokenizer, AutoConfig
import torch
import json
from huggingface_hub import hf_hub_download

# --- Updated Model Loading ---
pretrained_model_name = "frankmorales2020/FlightPlan_Transformer_LLM"

# 1. Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)

# 2. Load configuration using hf_hub_download
config_path = hf_hub_download(repo_id=pretrained_model_name, filename="config.json")
with open(config_path, 'r') as f:
    config_dict = json.load(f)

# *** Add max_waypoints to the config dictionary or load it from elsewhere ***
config_dict['max_waypoints'] = 10  # Replace with your actual max_waypoints value

# 3. Instantiate your SimpleTransformer class using the loaded configuration
model = SimpleTransformer(
    vocab_size=config_dict['vocab_size'],
    embed_dim=config_dict['embed_dim'],
    num_heads=config_dict['num_heads'],
    ff_hidden_dim=config_dict['ff_hidden_dim'],
    num_layers=config_dict['num_layers'],
    block_size=config_dict['block_size'],
    dropout=config_dict['dropout'],
    max_waypoints=config_dict['max_waypoints'] # Pass max_waypoints here
)

# 4. Load the model weights using hf_hub_download
model_path = hf_hub_download(repo_id=pretrained_model_name, filename="pytorch_model.bin")
state_dict = torch.load(model_path, map_location=torch.device('cpu'))
model.load_state_dict(state_dict)

In [ ]:
print(model)

In [ ]:
# --- Load and Preprocess Flight Plan Data ---
flight_plan_dataset = load_dataset("frankmorales2020/flight_plan_waypoints")

# Split into train and validation sets
flight_plan_dataset = flight_plan_dataset['train'].train_test_split(test_size=0.2, seed=42)
flight_plan_dataset = DatasetDict({
    'train': flight_plan_dataset['train'],
    'validation': flight_plan_dataset['test']
})

In [ ]:
flight_plan_dataset['train'][1]

In [ ]:
import random
import math
from datasets import Dataset
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
from tqdm import tqdm
from geopy.point import Point  # Import Point object
import warnings
warnings.filterwarnings("ignore")
import torch
import re

# Define airport codes and names
airports = [
    # North America
    "ATL", "LAX", "ORD", "DFW", "DEN", "JFK", "SFO", "LAS", "SEA", "CLT",
    "MIA", "PHL", "EWR", "BOS", "PHX", "MCO", "IAH", "DAL", "FLL", "DTW",
    "MSP", "IAD", "BWI", "SLC", "SAN", "MDW", "TPA", "PDX", "BNA", "STL",
    # Europe
    "LHR", "CDG", "AMS", "FRA", "MAD", "BCN", "FCO", "MUC", "DUB", "ZRH",
    # Asia
    "HND", "PVG", "ICN", "SIN", "KUL", "BKK", "DXB", "DEL", "HKG", "NRT",
    # Caribbean
    "CUN", "PUJ", "SJU", "MBJ", "NAS",
    # South America
    "GRU", "BOG", "EZE", "SCL", "LIM",
    # Australia & New Zealand
    "SYD", "MEL", "BNE", "PER", "AKL"
]

airport_names = {
    "ATL": "Hartsfield-Jackson Atlanta International Airport",
    "LAX": "Los Angeles International Airport",
    "ORD": "Chicago O'Hare International Airport",
    "DFW": "Dallas/Fort Worth International Airport",
    "DEN": "Denver International Airport",
    "JFK": "John F. Kennedy International Airport",
    "SFO": "San Francisco International Airport",
    "LAS": "Harry Reid International Airport",
    "SEA": "Seattle-Tacoma International Airport",
    "CLT": "Charlotte Douglas International Airport",
    "MIA": "Miami International Airport",
    "PHL": "Philadelphia International Airport",
    "EWR": "Newark Liberty International Airport",
    "BOS": "Logan International Airport",
    "PHX": "Phoenix Sky Harbor International Airport",
    "MCO": "Orlando International Airport",
    "IAH": "George Bush Intercontinental Airport",
    "DAL": "Dallas Love Field",
    "FLL": "Fort Lauderdale-Hollywood International Airport",
    "DTW": "Detroit Metropolitan Airport",
    "MSP": "Minneapolis-Saint Paul International Airport",
    "IAD": "Washington Dulles International Airport",
    "BWI": "Baltimore/Washington International Airport",
    "SLC": "Salt Lake City International Airport",
    "SAN": "San Diego International Airport",
    "MDW": "Midway International Airport",
    "TPA": "Tampa International Airport",
    "PDX": "Portland International Airport",
    "BNA": "Nashville International Airport",
    "STL": "St. Louis Lambert International Airport",
    # Europe
    "LHR": "London Heathrow Airport",
    "CDG": "Charles de Gaulle Airport",
    "AMS": "Amsterdam Airport Schiphol",
    "FRA": "Frankfurt Airport",
    "MAD": "Adolfo Suárez Madrid–Barajas Airport",
    "BCN": "Barcelona–El Prat Airport",
    "FCO": "Leonardo da Vinci–Fiumicino Airport",
    "MUC": "Munich Airport",
    "DUB": "Dublin Airport",
    "ZRH": "Zurich Airport",
    # Asia
    "HND": "Haneda Airport",
    "PVG": "Shanghai Pudong International Airport",
    "ICN": "Incheon International Airport",
    "SIN": "Singapore Changi Airport",
    "KUL": "Kuala Lumpur International Airport",
    "BKK": "Suvarnabhumi Airport",
    "DXB": "Dubai International Airport",
    "DEL": "Indira Gandhi International Airport",
    "HKG": "Hong Kong International Airport",
    "NRT": "Narita International Airport",
    # Caribbean
    "CUN": "Cancún International Airport",
    "PUJ": "Punta Cana International Airport",
    "SJU": "Luis Muñoz Marín International Airport",
    "MBJ": "Sangster International Airport",
    "NAS": "Lynden Pindling International Airport",
    # South America
    "GRU": "São Paulo–Guarulhos International Airport",
    "BOG": "El Dorado International Airport",
    "EZE": "Ministro Pistarini International Airport",
    "SCL": "Arturo Merino Benítez International Airport",
    "LIM": "Jorge Chávez International Airport",
    # Australia & New Zealand
    "SYD": "Sydney Airport",
    "MEL": "Melbourne Airport",
    "BNE": "Brisbane Airport",
    "PER": "Perth Airport",
    "AKL": "Auckland Airport"
}

In [ ]:
def generate_flight_plan(model, tokenizer, start_text, dataset, device='cuda'):
    """
    Generates a flight plan (list of waypoint names) based on the given input text.

    Args:
        model: The trained Transformer model.
        tokenizer: The tokenizer used for the model.
        start_text: The input text describing the flight plan request.
        dataset: The dataset used for training (to get num_waypoints).
        device: The device to run the model on (default: 'cuda').

    Returns:
        A list of waypoint names representing the generated flight plan.
    """
    model.eval()
    model.to(device)

    start_tokens = tokenizer(start_text, return_tensors='pt', padding=True, truncation=True).to(device)

    # Get num_waypoints from dataset based on partial match if exact match not found
    num_waypoints = None
    for example in dataset:
        if start_text.strip() in example['input'].strip():
            num_waypoints = example['label']
            break

    if num_waypoints is None:
        num_waypoints = 5  # Default value if not found in dataset
        print("Warning: Input text not found in dataset. Using default num_waypoints.")

    # Create a tensor for num_waypoints and move it to the device
    num_waypoints_tensor = torch.tensor([num_waypoints], device=device)

    with torch.no_grad():
        # Call the model with input_ids, attention_mask, and num_waypoints_tensor
        # Before calling the model, ensure all input_ids are within the vocabulary range
        start_tokens['input_ids'] = start_tokens['input_ids'].clip(0, tokenizer.vocab_size - 1)  # Clip to valid range

        predicted_waypoints = model(start_tokens['input_ids'], start_tokens['attention_mask'], num_waypoints_tensor)

    # Extract predicted airport codes directly, handling potential invalid codes
    predicted_codes = []
    for idx in predicted_waypoints[0].argmax(dim=-1):
        predicted_token = tokenizer.convert_ids_to_tokens(idx.item())

        # Check if the predicted token is a valid airport code, ignoring case
        if predicted_token.upper() in airports:
            predicted_codes.append(predicted_token.upper()) # Append in uppercase
        else:
            # Handle invalid airport codes (e.g., print a warning or use a default)
            print(f"Warning: Predicted token '{predicted_token}' is not a valid airport code. Skipping.")
            # You could append a default airport code here if needed

    # Limit to num_waypoints
    predicted_codes = predicted_codes[:num_waypoints]

    # Resolve airport codes to names
    waypoint_names = [airport_names.get(code, code) for code in predicted_codes]  # Use airport_names if available, otherwise use code

    # Extract origin and destination from start_text using regex
    match = re.search(r"from\s+([A-Z]{3})\s+to\s+([A-Z]{3})", start_text, re.IGNORECASE)
    if match:
        origin, destination = match.group(1), match.group(2)
        # Add origin and destination airport codes if not already present
        if waypoint_names and waypoint_names[0] != origin:
            waypoint_names.insert(0, origin)
        if waypoint_names and waypoint_names[-1] != destination:
            waypoint_names.append(destination)
    else:
        print("Warning: Could not extract origin and destination from input text.")

    return waypoint_names

In [ ]:
#query='Calculate the waypoints from DXB to MSP. Departure: 2024-05-03, Aircraft: Boeing 777, Weather: Overcast'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
query= 'Calculate the waypoints from LHR to AMS. Departure: 2024-06-15, Aircraft: Piper PA-28, Weather: Partly Cloudy'
print(f"Query: {query}")
generated_plan = generate_flight_plan(model, tokenizer, start_text=query, dataset=flight_plan_dataset['train'], device=device) # Pass the dataset here
print("\nGenerated waypoint list:")
print(generated_plan)